In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"]="2,3"

In [2]:
from module.load_data import load_npz

# Load data
dataset_main_path = 'formatted_dataset/face_recognition/'

pair_1_fname_train, pair_2_fname_train, label_pairing_train, label_anchor_train, label_non_anchor_train = load_npz(dataset_main_path+"train.npz")
pair_1_fname_test, pair_2_fname_test, label_pairing_test, label_anchor_test, label_non_anchor_test = load_npz(dataset_main_path+"test.npz")
#pair_1_fname_val, pair_2_fname_val, label_pairing_val, label_anchor_val, label_non_anchor_val = load_npz(dataset_main_path+"val.npz")

In [3]:
# import matplotlib.pyplot as plt

# idx = 0
# print(pair_1_fname_train[idx])
# print(label_pairing_train[idx])
# print(label_anchor_train[idx])
# print(pair_2_fname_train[idx])
# print(label_non_anchor_train[idx])

In [4]:
# import cv2

# plt.imshow(cv2.imread("dataset/cifar10_randomed/images/" + pair_1_fname_train[idx]))

In [5]:
# plt.imshow(cv2.imread("dataset/cifar10_randomed/images/" + pair_2_fname_train[idx]))

In [6]:
import tqdm

from module.util_llm import *
from module.inference_llm import *

def get_url_string(fname_list_1:str, fname_list_2:str, image_base_dir:str='dataset/cifar10_randomed/images'):
    url_1_list, url_2_list = [], []
    for idx, fname in tqdm.tqdm(enumerate(fname_list_1)):
        url_1 = pixtral_preprocessing(
            os.path.join(image_base_dir, fname_list_1[idx])
        )
        
        url_2 = pixtral_preprocessing(
            os.path.join(image_base_dir, fname_list_2[idx])
        )
        
        url_1_list.append(url_1)
        url_2_list.append(url_2)
        
    return url_1_list, url_2_list

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-05 05:44:42.904645: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-05 05:44:42.918121: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-05 05:44:42.922356: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-05 05:44:42.932810: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to 

In [7]:
pair_1_url_train, pair_2_url_train = get_url_string(
    fname_list_1=pair_1_fname_train,
    fname_list_2=pair_2_fname_train
)

pair_1_url_test, pair_2_url_test = get_url_string(
    fname_list_1=pair_1_fname_test,
    fname_list_2=pair_2_fname_test
)

22032it [00:00, 26732.65it/s]
8204it [00:00, 26993.52it/s]


In [8]:
model, sampling_params = load_model(model_name='mistralai/Pixtral-12B-2409')

INFO 11-05 05:44:48 config.py:1664] Downcasting torch.float32 to torch.float16.
INFO 11-05 05:44:56 llm_engine.py:237] Initializing an LLM engine (v0.6.3.post1) with config: model='mistralai/Pixtral-12B-2409', speculative_config=None, tokenizer='mistralai/Pixtral-12B-2409', skip_tokenizer_init=False, tokenizer_mode=mistral, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=mistralai/Pixtral-12B-2409, num

/home/islab-ai/.local/lib/python3.10/site-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_fwd")
/home/islab-ai/.local/lib/python3.10/site-packages/xformers/ops/fmha/flash.py:344: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_bwd")


INFO 11-05 05:45:00 weight_utils.py:243] Using model weights format ['*.safetensors']
INFO 11-05 05:45:00 weight_utils.py:288] No model.safetensors.index.json found in remote.


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:05<00:00,  5.22s/it]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:05<00:00,  5.22s/it]



INFO 11-05 05:45:07 model_runner.py:1067] Loading model weights took 23.6552 GB
INFO 11-05 05:45:13 gpu_executor.py:122] # GPU blocks: 17690, # CPU blocks: 1638
INFO 11-05 05:45:13 gpu_executor.py:126] Maximum concurrency for 32768 tokens per request: 8.64x
INFO 11-05 05:45:15 model_runner.py:1395] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 11-05 05:45:15 model_runner.py:1399] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 11-05 05:45:36 model_runner.py:1523] Graph capturing finished in 21 secs.


In [9]:
prompt = 'Describe the person you saw from the given images especially from hair, face, and nose. Tell only the differences or similarity without any chat'

In [ ]:
pair_text_train = pixtral_inference(
    llm=model,
    sampling_params=sampling_params,
    image_url_1=pair_1_url_train,
    image_url_2=pair_2_url_train,
    prompt=prompt
)

# Save into json based on fname
export_generated_text_json(
    generated_text=pair_text_train,
    fname_list_pair_1=pair_1_fname_train, 
    fname_list_pair_2=pair_2_fname_train,
    target_dir='formatted_dataset/face_recognition/text'
)

0it [00:00, ?it/s]

WARNING 11-05 05:45:36 chat_utils.py:570] 'add_generation_prompt' is not supported for mistral tokenizer, so it will be ignored.
WARNING 11-05 05:45:36 chat_utils.py:574] 'continue_final_message' is not supported for mistral tokenizer, so it will be ignored.


1it [00:02,  2.99s/it]

WARNING 11-05 05:45:39 chat_utils.py:570] 'add_generation_prompt' is not supported for mistral tokenizer, so it will be ignored.
WARNING 11-05 05:45:39 chat_utils.py:574] 'continue_final_message' is not supported for mistral tokenizer, so it will be ignored.


2it [00:06,  3.02s/it]

...

WARNING 11-05 10:59:40 chat_utils.py:570] 'add_generation_prompt' is not supported for mistral tokenizer, so it will be ignored.
WARNING 11-05 10:59:40 chat_utils.py:574] 'continue_final_message' is not supported for mistral tokenizer, so it will be ignored.


7739it [5:14:05,  1.97s/it]

WARNING 11-05 10:59:42 chat_utils.py:570] 'add_generation_prompt' is not supported for mistral tokenizer, so it will be ignored.
WARNING 11-05 10:59:42 chat_utils.py:574] 'continue_final_message' is not supported for mistral tokenizer, so it will be ignored.


7740it [5:14:09,  2.51s/it]

WARNING 11-05 10:59:46 chat_utils.py:570] 'add_generation_prompt' is not supported for mistral tokenizer, so it will be ignored.


In [ ]:
pair_text_test = pixtral_inference(
    llm=model,
    sampling_params=sampling_params,
    image_url_1=pair_1_url_test
    image_url_2=pair_2_url_test
    prompt=prompt
)

export_generated_text_json(
    generated_text=pair_text_test,
    fname_list_pair_1=pair_1_fname_test,
    fname_list_pair_2=pair_2_fname_test,
    target_dir='formatted_dataset/face_recognition/text'
)

In [ ]:
#pair_text[0][0].outputs[0].text

In [ ]:
# import matplotlib.pyplot as plt

# from module.load_data import *

# idx = 1
# base_image_path = 'formatted_dataset/face_recognition/images/'
# image_1, image_2 = load_image_pair(base_image_path + pair_1_fname_train[idx], base_image_path + pair_2_fname_train[idx])
# print(label_pairing_train[0], label_anchor_train[0], label_non_anchor_train[0])

In [ ]:
# plt.imshow(image_1)

In [ ]:
# plt.imshow(image_2)